Here we use swap test to measure the inner product between the two states.

In [2]:
import pennylane as qml
from pennylane import numpy as np

In [23]:
dev = qml.device("default.qubit", wires = 3)
np.random.seed(1)

In [10]:
alpha = 1 / np.sqrt(2)
beta = 1 / np.sqrt(2)
x = np.array([alpha, beta])

In [29]:
@qml.qnode(dev)
def swap_test_model(params):
    qml.QubitStateVector(x, wires = 1)
    qml.RX(params[0], wires = 2)
    qml.RY(params[1], wires = 2)
    
    qml.Hadamard(wires = 0)
    qml.CSWAP(wires = [0,1,2])
    qml.Hadamard(wires = 0)
    return qml.expval(qml.PauliZ(0))

In [37]:
def swap_test_train(steps):
    opt = qml.AdamOptimizer(stepsize = 0.1)
    
    params = np.random.random(2)
    
    for i in range(steps):
        params, prev_energy = opt.step_and_cost(swap_test_model, params)     
        if i % 10 == 0:
            print(prev_energy)    
    return params

In [39]:
params = swap_test_train(100)
swap_test_model(params)

0.8767551804714175
0.5820879668990511
0.47976381666652723
0.4001401879733402
0.1385774999602699
0.02690369734876663
0.0060624813990338144
0.009130194023125093
0.00036133257721315815
0.0007425352186824496


tensor(0.00034217, requires_grad=True)